<a href="https://colab.research.google.com/github/Kaanxx/EmpSim/blob/main/EmpSim_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Camel AI için gerekli paketleri kurun ya da direkt olarak OWL'u github'dan çekebilirsiniz.
!pip install camel-ai
!pip install colorama python-dotenv -q

In [ ]:
#@title RolePlay ve Agentmodların importu. Tool eklemek için (web_search, wikipedia, x, reddit vs. camel.toolkit import SearchToolKit" gibi ekleme yapabilirsiniz.)
import os
import json
from typing import List, Dict, Any, Tuple
from datetime import datetime
import time
from colorama import Fore, Style, init

# CAMEL AI imports
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.messages import BaseMessage
from camel.societies import RolePlaying
from camel.utils import print_text_animated

# Colorama'yı başlat
init(autoreset=True)

In [ ]:
os.environ["GEMINI_API_KEY"] = "KENDİ_API_ANAHTARINIZ(BAŞKA ENDPOINTLER İÇİN DEĞİŞTİRİN)"

In [ ]:
#@title Çalışanlar sınıfı burada yer almasını istediğiniz kişileri direkt olarak JSONL formatında ekleyeiblirsiniz dilerseniz structered_output ile başka LLM'lerden karakter oluşturmasını isteyeiblirsiniz.
CALISANLAR = [
    {
        "isim": "Ahmet",
        "rol": "Proje Yöneticisi",
        "karakter": """Detaycı, organize, takım çalışmasına önem veren bir proje yöneticisisin.
        Her zaman deadline'lara dikkat edersin ve projenin genel görünümünü takip edersin.
        Düşünce tarzın: Önce büyük resmi görürsün, sonra detaylara inersin.""",
        "dusunme_stili": "analitik ve sistematik"
    },
    {
        "isim": "Zeynep",
        "rol": "Yazılım Geliştirici",
        "karakter": """Yaratıcı, problem çözme yeteneği yüksek bir yazılımcısın.
        Yeni teknolojilere meraklısın ve kod kalitesine önem verirsin.
        Düşünce tarzın: Önce problemi analiz edersin, sonra en verimli çözümü ararsın.""",
        "dusunme_stili": "mantıksal ve yenilikçi"
    },
    {
        "isim": "Mehmet",
        "rol": "Satış Müdürü",
        "karakter": """İkna kabiliyeti yüksek, müşteri odaklı bir satış müdürüsün.
        Hedef odaklısın ve her zaman satış stratejileri geliştirirsin.
        Düşünce tarzın: Müşteri ihtiyaçlarını öncelikli düşünürsün.""",
        "dusunme_stili": "stratejik ve sonuç odaklı"
    },
    {
        "isim": "Ayşe",
        "rol": "İnsan Kaynakları Uzmanı",
        "karakter": """Empati yeteneği yüksek, çalışan mutluluğunu önemseyen bir İK uzmanısın.
        Takım dinamiklerini iyi anlarsın ve çatışma çözümünde başarılısın.
        Düşünce tarzın: İnsan faktörünü her zaman ön planda tutarsın.""",
        "dusunme_stili": "empatik ve dengeli"
    },
    {
        "isim": "Can",
        "rol": "Pazarlama Uzmanı",
        "karakter": """Kreatif, trend takipçisi ve sosyal medya uzmanı bir pazarlamacısın.
        Marka bilinirliğini artırmak için yenilikçi fikirler üretirsin.
        Düşünce tarzın: Hedef kitleyi analiz eder ve yaratıcı çözümler bulursun.""",
        "dusunme_stili": "yaratıcı ve analitik"
    },
    {
        "isim": "Fatma",
        "rol": "Muhasebe Müdürü",
        "karakter": """Detaycı, kurallara bağlı ve sayılarla arası iyi bir muhasebecisisin.
        Finansal raporları titizlikle hazırlarsın ve bütçe kontrolünde ustasın.
        Düşünce tarzın: Her şeyi sayılarla değerlendirir ve risk analizine önem verirsin.""",
        "dusunme_stili": "analitik ve risk odaklı"
    },
    {
        "isim": "Burak",
        "rol": "Ürün Tasarımcısı",
        "karakter": """Estetik anlayışı güçlü, kullanıcı deneyimine önem veren bir tasarımcısın.
        Minimalist ve fonksiyonel tasarımlar yapmayı seversin.
        Düşünce tarzın: Kullanıcı gözünden bakar ve estetikle fonksiyonelliği birleştirirsin.""",
        "dusunme_stili": "görsel ve kullanıcı odaklı"
    },
    {
        "isim": "Selin",
        "rol": "Veri Analisti",
        "karakter": """Veri odaklı, analitik düşünen ve içgörü çıkarmada başarılı bir analistsin.
        Karmaşık verileri anlamlı hikayelere dönüştürürsün.
        Düşünce tarzın: Veriler konuşur, sen dinler ve yorumlarsın.""",
        "dusunme_stili": "veri odaklı ve objektif"
    },
    {
        "isim": "Emre",
        "rol": "Müşteri Hizmetleri Yöneticisi",
        "karakter": """Sabırlı, çözüm odaklı ve müşteri memnuniyetini önceliğe alan bir yöneticisin.
        Zor müşterilerle bile profesyonel iletişim kurarsın.
        Düşünce tarzın: Her müşteri sorunu bir gelişim fırsatıdır.""",
        "dusunme_stili": "çözüm odaklı ve sakin"
    },
    {
        "isim": "Deniz",
        "rol": "Operasyon Müdürü",
        "karakter": """Süreç optimizasyonu konusunda uzman, verimlilik odaklı bir yöneticisin.
        Operasyonel mükemmelliği hedeflersin ve sürekli iyileştirme ararsın.
        Düşünce tarzın: Her süreç daha iyi hale getirilebilir.""",
        "dusunme_stili": "süreç odaklı ve verimlilik düşkünü"
    }
]


In [ ]:
#@title Gemini-2.5-flash modeli hem token hem multimodal seçeniğiyle en f/p tercih. dilerseniz openai uyumluluğu olan sdkler ile değiştirebilrisiniz (chatgpt,qwen,huggingface vs gibi)
def create_gemini_model(temperature=0.7, max_tokens=65536):
    """Gemini 2.5 Flash modelini oluşturur"""
    try:
        model = ModelFactory.create(
            model_platform=ModelPlatformType.OPENAI_COMPATIBLE_MODEL,
            model_type="gemini-2.5-flash",
            api_key=os.getenv("GEMINI_API_KEY"),
            url="https://generativelanguage.googleapis.com/v1beta/openai/",
            model_config_dict={
                "temperature": temperature,
                "max_tokens": max_tokens
            },
        )
        return model
    except Exception as e:
        print(f"❌ Model oluşturma hatası: {e}")
        print("Lütfen GEMINI_API_KEY'i kontrol edin!")
        return None

In [ ]:
#@title Burada Agent'a rol ataması yapıyoruz.
class CalisanAgent:
    def __init__(self, calisan_bilgi: Dict[str, str]):
        self.isim = calisan_bilgi["isim"]
        self.rol = calisan_bilgi["rol"]
        self.karakter = calisan_bilgi["karakter"]
        self.dusunme_stili = calisan_bilgi["dusunme_stili"]

        # Sistem mesajını oluştur
        system_message = f"""Sen {self.isim} isimli bir {self.rol}'sın.

        Karakter Özelliklerin:
        {self.karakter}

        Düşünme Stilin: {self.dusunme_stili}

        Görev aldığında:
        1. Önce düşünce sürecini aç (DÜŞÜNÜYORUM: ... şeklinde)
        2. Görevi kendi bakış açından değerlendir
        3. Rolüne ve karakterine uygun bir yaklaşım geliştir
        4. Çözüm önerini veya görüşünü sun

        Her zaman karakterine sadık kal ve düşünce sürecini göster."""

        # Agent'ı oluştur
        model = create_gemini_model(temperature=0.8)
        if model:
            self.agent = ChatAgent(
                system_message=system_message,
                model=model
            )
        else:
            self.agent = None

    def gorev_yap(self, gorev: str) -> str:
        """Verilen görevi yapar ve yanıt döner"""
        if not self.agent:
            return f"{self.isim}: Model bağlantı hatası!"

        try:
            response = self.agent.step(gorev)
            return f"{response.msgs[0].content}"
        except Exception as e:
            return f"{self.isim}: Yanıt verirken hata oluştu - {str(e)}"

In [ ]:
#@title Simülasyon adımları
class IsSimulasyonu:
    def __init__(self, calisanlar_listesi: List[Dict[str, str]]):
        """İş simülasyonunu başlatır"""
        print(Fore.CYAN + "=" * 60)
        print(Fore.YELLOW + "🏢 İŞ SİMÜLASYONU SİSTEMİ BAŞLATILIYOR...")
        print(Fore.CYAN + "=" * 60)

        self.calisanlar = []
        for calisan in calisanlar_listesi:
            print(f"⚙️ {calisan['isim']} ({calisan['rol']}) oluşturuluyor...")
            agent = CalisanAgent(calisan)
            self.calisanlar.append(agent)
            time.sleep(0.5)

        print(Fore.GREEN + "\n✅ Tüm çalışanlar hazır!\n")

    def gorev_ata(self, gorev: str, secilen_calisanlar: List[str] = None):
        """Görevi seçilen çalışanlara atar"""
        print(Fore.MAGENTA + "=" * 60)
        print(Fore.YELLOW + f"📋 YENİ GÖREV: {gorev}")
        print(Fore.MAGENTA + "=" * 60 + "\n")

        # Eğer çalışan seçilmemişse hepsine ata
        if secilen_calisanlar is None:
            hedef_calisanlar = self.calisanlar
        else:
            hedef_calisanlar = [c for c in self.calisanlar if c.isim in secilen_calisanlar]

        yanitlar = []
        for calisan in hedef_calisanlar:
            print(Fore.CYAN + f"\n{'='*50}")
            print(Fore.GREEN + f"👤 {calisan.isim} ({calisan.rol}) yanıtlıyor...")
            print(Fore.CYAN + f"{'='*50}")

            yanit = calisan.gorev_yap(gorev)

            # Düşünce sürecini vurgula
            if "DÜŞÜNÜYORUM:" in yanit:
                parts = yanit.split("DÜŞÜNÜYORUM:")
                print(Fore.YELLOW + "💭 DÜŞÜNCE SÜRECİ:")
                if len(parts) > 1:
                    dusunce_kismi = parts[1].split("\n")[0]
                    print(Fore.WHITE + dusunce_kismi)
                    geri_kalan = "\n".join(parts[1].split("\n")[1:])
                    print(Fore.WHITE + "\n📝 YANIT:" + geri_kalan)
            else:
                print(Fore.WHITE + yanit)

            yanitlar.append({
                "calisan": calisan.isim,
                "rol": calisan.rol,
                "yanit": yanit
            })

            time.sleep(1)

        return yanitlar

    def takım_toplantısı(self, konu: str):
        """Takım toplantısı simülasyonu"""
        print(Fore.MAGENTA + "\n" + "=" * 60)
        print(Fore.YELLOW + f"🤝 TAKIM TOPLANTISI: {konu}")
        print(Fore.MAGENTA + "=" * 60 + "\n")

        # Önce proje yöneticisi konuyu açar
        pm = next((c for c in self.calisanlar if c.rol == "Proje Yöneticisi"), None)
        if pm:
            print(Fore.GREEN + f"\n👔 {pm.isim} toplantıyı açıyor...")
            acilis = pm.gorev_yap(f"Takım toplantısı açılışı yap. Konu: {konu}")
            print(Fore.WHITE + acilis)

        # Diğer çalışanlar görüş bildirir
        for calisan in self.calisanlar:
            if calisan.rol != "Proje Yöneticisi":
                print(Fore.CYAN + f"\n💬 {calisan.isim} ({calisan.rol}) görüş bildiriyor...")
                gorus = calisan.gorev_yap(
                    f"'{konu}' konusundaki takım toplantısında kendi rolün açısından görüşünü bildir. Kısa ve öz ol."
                )
                print(Fore.WHITE + gorus[:500] + "..." if len(gorus) > 500 else gorus)
                time.sleep(0.5)

In [ ]:
#@title Çalışanların hangi senaryoda nasıl davranacağını simüle etmek için senaryolar oluşturuyoruz burayı kendi use_case'lerinzie göre dizayn edebilirsiniz.
def main():
    """Ana simülasyon fonksiyonu"""

    # Simülasyonu başlat
    print(Fore.YELLOW + "\n🚀 İŞ SİMÜLASYONU BAŞLIYOR!\n")
    simulasyon = IsSimulasyonu(CALISANLAR)

    # SENARYO 1: Genel görev
    print(Fore.MAGENTA + "\n" + "=" * 60)
    print(Fore.YELLOW + "SENARYO 1: Yeni Ürün Lansmanı")
    print(Fore.MAGENTA + "=" * 60)

    gorev1 = """Şirketimiz yeni bir mobil uygulama geliştirdi.
    Bu uygulamanın piyasaya sürülmesi için kendi departmanınızın perspektifinden
    ne tür hazırlıklar yapılması gerektiğini ve dikkat edilmesi gereken noktaları belirtin."""

    # Sadece ilgili departmanlara ata
    ilgili_calisanlar = ["Ahmet", "Zeynep", "Can", "Burak", "Mehmet"]
    yanitlar1 = simulasyon.gorev_ata(gorev1, ilgili_calisanlar)

    # SENARYO 2: Kriz yönetimi
    print(Fore.MAGENTA + "\n" + "=" * 60)
    print(Fore.YELLOW + "SENARYO 2: Müşteri Şikayeti Krizi")
    print(Fore.MAGENTA + "=" * 60)

    gorev2 = """Sosyal medyada ürünümüzle ilgili viral olan olumsuz bir yorum var.
    Bu durumu nasıl yönetmeliyiz? Acil aksiyon planı öner."""

    kriz_ekibi = ["Can", "Emre", "Mehmet", "Ayşe"]
    yanitlar2 = simulasyon.gorev_ata(gorev2, kriz_ekibi)

    # SENARYO 3: Takım toplantısı
    simulasyon.takım_toplantısı("Q2 hedeflerimiz ve departmanlar arası koordinasyon")

    print(Fore.GREEN + "\n" + "=" * 60)
    print(Fore.YELLOW + "✅ SİMÜLASYON TAMAMLANDI!")
    print(Fore.GREEN + "=" * 60)


In [ ]:
#@title Süreç Bittikten sonra devam edip tüm ekip başka bir göreve geçiş için E'ye tıkladıktan sonra görev atamasını yapıyoruz.
def ozel_gorev_ata(simulasyon: IsSimulasyonu):
    """Kullanıcıdan özel görev alır ve atar"""
    print(Fore.YELLOW + "\n📝 ÖZEL GÖREV ATAMA")
    print(Fore.CYAN + "-" * 40)

    # Çalışanları listele
    print("\nMevcut Çalışanlar:")
    for i, calisan in enumerate(simulasyon.calisanlar, 1):
        print(f"{i}. {calisan.isim} - {calisan.rol}")

    # Görev al
    gorev = input(Fore.YELLOW + "\nGörev açıklamasını girin: ")

    # Çalışan seçimi
    secim = input(Fore.YELLOW + "Tüm çalışanlara mı atansın? (E/H): ").upper()

    if secim == "H":
        secilen_isimler = input("Çalışan isimlerini virgülle ayırarak girin: ").split(",")
        secilen_isimler = [isim.strip() for isim in secilen_isimler]
        simulasyon.gorev_ata(gorev, secilen_isimler)
    else:
        simulasyon.gorev_ata(gorev)

In [ ]:
#@title özel görev ataması için E'ye basıp devam edebilirsiniz.
if __name__ == "__main__":
    # API Key kontrolü
    if os.getenv("GEMINI_API_KEY") == "YOUR_GEMINI_API_KEY_HERE":
        print(Fore.RED + "⚠️ UYARI: Lütfen GEMINI_API_KEY'inizi ayarlayın!")
        print(Fore.YELLOW + "Step 3'teki 'YOUR_GEMINI_API_KEY_HERE' kısmını değiştirin.")
    else:
        try:
            # Ana simülasyonu çalıştır
            main()

            # İsteğe bağlı: Özel görev atama
            print(Fore.YELLOW + "\n" + "=" * 60)
            devam = input("Özel görev atamak ister misiniz? (E/H): ").upper()
            if devam == "E":
                simulasyon = IsSimulasyonu(CALISANLAR)
                ozel_gorev_ata(simulasyon)

        except Exception as e:
            print(Fore.RED + f"\n❌ Hata oluştu: {e}")
            print(Fore.YELLOW + "Lütfen API key'inizi ve internet bağlantınızı kontrol edin.")

# =============================================
# NOTLAR VE KULLANIM TALİMATLARI
# =============================================
"""
📌 KULLANIM TALİMATLARI:

1. Google Colab'de yeni bir notebook açın
2. Bu kodu hücrelere bölerek yapıştırın (STEP'lere göre)
3. İlk önce STEP 1'i çalıştırarak kütüphaneleri yükleyin
4. STEP 3'te kendi Gemini API anahtarınızı girin
5. STEP 4'te çalışan karakterlerini istediğiniz gibi düzenleyin
6. Tüm hücreleri sırayla çalıştırın

📌 ÖZELLEŞTİRME SEÇENEKLERİ:

1. ÇALIŞAN KARAKTERLERİ (STEP 4):
   - İsim, rol, karakter ve düşünme stillerini değiştirebilirsiniz
   - Yeni çalışanlar ekleyebilir veya mevcut olanları silebilirsiniz

2. MODEL AYARLARI (STEP 5):
   - temperature: Yaratıcılık seviyesi (0.0-1.0)
   - max_tokens: Maksimum yanıt uzunluğu

3. GÖREVLER (STEP 8):
   - main() fonksiyonunda yeni senaryolar ekleyebilirsiniz
   - ozel_gorev_ata() ile interaktif görev atayabilirsiniz

📌 ÖNEMLİ NOTLAR:
- Gemini API anahtarınız olmadan sistem çalışmaz
- Her agent çağrısı API kotanızdan düşer
- İnternet bağlantısı gereklidir
"""


🚀 İŞ SİMÜLASYONU BAŞLIYOR!

🏢 İŞ SİMÜLASYONU SİSTEMİ BAŞLATILIYOR...
⚙️ Ahmet (Proje Yöneticisi) oluşturuluyor...
⚙️ Zeynep (Yazılım Geliştirici) oluşturuluyor...
⚙️ Mehmet (Satış Müdürü) oluşturuluyor...
⚙️ Ayşe (İnsan Kaynakları Uzmanı) oluşturuluyor...
⚙️ Can (Pazarlama Uzmanı) oluşturuluyor...
⚙️ Fatma (Muhasebe Müdürü) oluşturuluyor...
⚙️ Burak (Ürün Tasarımcısı) oluşturuluyor...
⚙️ Selin (Veri Analisti) oluşturuluyor...
⚙️ Emre (Müşteri Hizmetleri Yöneticisi) oluşturuluyor...
⚙️ Deniz (Operasyon Müdürü) oluşturuluyor...

✅ Tüm çalışanlar hazır!


SENARYO 1: Yeni Ürün Lansmanı
📋 YENİ GÖREV: Şirketimiz yeni bir mobil uygulama geliştirdi. 
    Bu uygulamanın piyasaya sürülmesi için kendi departmanınızın perspektifinden 
    ne tür hazırlıklar yapılması gerektiğini ve dikkat edilmesi gereken noktaları belirtin.


👤 Ahmet (Proje Yöneticisi) yanıtlıyor...
💭 DÜŞÜNCE SÜRECİ:
 Şirketimizin yeni mobil uygulamasının piyasaya sürülmesi, projenin en kritik aşamalarından biridir. Bu, sadece t